In [171]:
import numpy as np

from enum import Enum
from typing import List


In [172]:
class Signs(Enum):
    LESS = 1
    EQUAL = 2
    GREAT = 3


class TaskType(Enum):
    MIN = 1
    MAX = 2


class Inequality:
    def __init__(self, coeff: List[int], sign: Signs, free: int):
        self.coeff = coeff
        self.sign = sign
        self.free = free


class Task:
    def __init__(self, main_coeff: List[int], task_type: TaskType, inequals: List[Inequality]):
        self.main_coeff = main_coeff
        self.task_type = task_type
        self.inequals = inequals


In [173]:
def check(actual, expected):
    if actual != expected:
        print(f'not equal. expected: {expected}, got: {actual}')
        # raise RuntimeError(f'not equal. expected: {expected}, got: {actual}')


In [174]:
def solve(task: Task, initial_solution: List[int], initial_basis: List[int]):
    begin_size = len(task.main_coeff)
    data_list = []
    print(len(task.inequals))
    for ineq_index in range(len(task.inequals)):
        if task.inequals[ineq_index].sign == Signs.LESS:
            task.inequals[ineq_index].coeff.append(1)
            task.main_coeff.append(0)
            for ineq_index2 in range(len(task.inequals)):
                if ineq_index != ineq_index2:
                    task.inequals[ineq_index2].coeff.append(0)
        elif task.inequals[ineq_index].sign == Signs.GREAT:
            task.inequals[ineq_index].coeff.append(-1)
            task.main_coeff.append(0)
            for ineq_index2 in range(len(task.inequals)):
                if ineq_index != ineq_index2:
                    task.inequals[ineq_index2].coeff.append(0)

    for ineq in task.inequals:
        # if ineq.sign != Signs.EQUAL:
        #     raise RuntimeError("< or > not supported yet")
        data_list.append(ineq.coeff + [-ineq.free])

    data = np.array(
        data_list,
        dtype=np.float64
    )

    result = np.array(task.main_coeff + [0], dtype=np.float64)

    if task.task_type == TaskType.MIN:
        result *= -1

    print(data.shape)

    n = data.shape[1] - 1
    m = data.shape[0]

    print(data)
    print(result)

    cnt = 0

    for ind in range(n):
        if initial_basis[ind] == 1:
            data[cnt] /= data[cnt][ind]
            for i in range(m):
                if i == cnt:
                    continue
                data[i] -= data[cnt] * data[i][ind]
            cnt += 1

    cnt = 0
    for ind in range(n):
        if initial_basis[ind] == 1:
            result -= data[cnt] * (result[ind] / data[cnt][ind])
            cnt += 1

    print(data)
    print(result)

    for cyc in range(100):
        ind_in_basis = -1
        for i in range(n):
            if result[i] > 0:
                ind_in_basis = i
                break
        if ind_in_basis == -1:
            print('complete')
            break
        ind_out_basis = -1
        prev_rez = -1e10
        for i in range(m):
            if data[i][ind_in_basis] == 0:
                continue
            tt1 = data[i][-1] / data[i][ind_in_basis]
            if 0 >= tt1 > prev_rez:
                ind_out_basis = i
                prev_rez = tt1
        if ind_out_basis == -1:
            print('unlimited')
            break
        # in_basis[ind_out_basis] = 0
        # in_basis[ind_in_basis] = 1

        data[ind_out_basis] /= data[ind_out_basis][ind_in_basis]
        for i in range(m):
            if i != ind_out_basis:
                data[i] -= data[ind_out_basis] * data[i][ind_in_basis]

        result -= data[ind_out_basis] * result[ind_in_basis]

        print(cyc)
        print(ind_in_basis, ind_out_basis)
        print(data)
        print(result)

    result_mn = 1 if task.task_type == TaskType.MAX else -1
    print("Result: ", result_mn * result[-1])

    return result_mn * result[-1]
    # print("In point: ")


In [175]:
task_1 = Task(
    [-6, -1, -4, 5], TaskType.MIN,
    [
        Inequality([3, 1, -1, 1], Signs.EQUAL, 4),
        Inequality([5, 1, 1, -1], Signs.EQUAL, 4)
    ]
)

result_1 = solve(
    task_1,
    [1, 0, 0, 1],
    [1, 0, 0, 1]
)

check(result_1, -4)


2
(2, 5)
[[ 3.  1. -1.  1. -4.]
 [ 5.  1.  1. -1. -4.]]
[ 6.  1.  4. -5. -0.]
[[ 1.    0.25  0.    0.   -1.  ]
 [-0.    0.25 -1.    1.   -1.  ]]
[ 0.    0.75 -1.    0.    1.  ]
0
1 0
[[ 4.  1.  0.  0. -4.]
 [-1.  0. -1.  1.  0.]]
[-3.  0. -1.  0.  4.]
complete
Result:  -4.0


In [176]:
task_2 = Task(
    [-1, -2, -3, 1], TaskType.MIN,
    [
        Inequality([1, -3, -1, -2], Signs.EQUAL, -4),
        Inequality([1, -1, 1, 0], Signs.EQUAL, 0)
    ]
)

result_2 = solve(
    task_2,
    [0, 1, 1, 0],
    [0, 1, 1, 0]
)

check(result_2, -6)


2
(2, 5)
[[ 1. -3. -1. -2.  4.]
 [ 1. -1.  1.  0.  0.]]
[ 1.  2.  3. -1. -0.]
[[-0.5  1.   0.   0.5 -1. ]
 [ 0.5  0.   1.   0.5 -1. ]]
[ 0.5  0.   0.  -3.5  5. ]
0
0 1
[[ 0.  1.  1.  1. -2.]
 [ 1.  0.  2.  1. -2.]]
[ 0.  0. -1. -4.  6.]
complete
Result:  -5.999999999999999
not equal. expected: -6, got: -5.999999999999999


In [177]:
task_2_max = Task(
    [-1, -2, -3, 1], TaskType.MAX,
    [
        Inequality([1, -3, -1, -2], Signs.EQUAL, -4),
        Inequality([1, -1, 1, 0], Signs.EQUAL, 0)
    ]
)

result_2_max = solve(
    task_2_max,
    [0, 1, 1, 0],
    [0, 1, 1, 0]
)

check(result_2_max, 2)


2
(2, 5)
[[ 1. -3. -1. -2.  4.]
 [ 1. -1.  1.  0.  0.]]
[-1. -2. -3.  1.  0.]
[[-0.5  1.   0.   0.5 -1. ]
 [ 0.5  0.   1.   0.5 -1. ]]
[-0.5  0.   0.   3.5 -5. ]
0
3 0
[[-1.  2.  0.  1. -2.]
 [ 1. -1.  1.  0.  0.]]
[ 3. -7.  0.  0.  2.]
1
0 1
[[ 0.  1.  1.  1. -2.]
 [ 1. -1.  1.  0.  0.]]
[ 0. -4. -3.  0.  2.]
complete
Result:  2.0


In [178]:
# TODO fix gauss

task_3 = Task(
    [-1, -2, -1, 3, -1], TaskType.MIN,
    [
        Inequality([0, 1, 1, 2, 1], Signs.EQUAL, 6),
        Inequality([1, 1, 0, 2, 1], Signs.EQUAL, 5),
        Inequality([1, 1, 1, 3, 2], Signs.EQUAL, 9)
    ]
)

result_3 = solve(
    task_3,
    [0, 0, 1, 2, 1],
    [0, 0, 1, 1, 1]
)

check(result_3, -11)

3
(3, 6)
[[ 0.  1.  1.  2.  1. -6.]
 [ 1.  1.  0.  2.  1. -5.]
 [ 1.  1.  1.  3.  2. -9.]]
[ 1.  2.  1. -3.  1. -0.]
[[-1.  0.  1.  0.  0. -1.]
 [ 0.  1.  0.  1.  0. -2.]
 [ 1. -1.  0.  0.  1. -1.]]
[ 1.  6.  0.  0.  0. -4.]
0
0 2
[[ 0. -1.  1.  0.  1. -2.]
 [ 0.  1.  0.  1.  0. -2.]
 [ 1. -1.  0.  0.  1. -1.]]
[ 0.  7.  0.  0. -1. -3.]
1
1 1
[[ 0.  0.  1.  1.  1. -4.]
 [ 0.  1.  0.  1.  0. -2.]
 [ 1.  0.  0.  1.  1. -3.]]
[ 0.  0.  0. -7. -1. 11.]
complete
Result:  -11.0


In [179]:
task4 = Task(
    [-1, -1, -1, 1, -1], TaskType.MIN,
    [
        Inequality([1, 1, 2, 0, 0], Signs.EQUAL, 4),
        Inequality([0, -2, -2, 1, -1], Signs.EQUAL, -6),
        Inequality([1, -1, 6, 1, 1], Signs.EQUAL, 12)
    ]
)

result_4 = solve(
    task4,
    [1, 1, 2, 0, 0],
    [1, 1, 1, 0, 0],
)

check(result_4, -10)

3
(3, 6)
[[  1.   1.   2.   0.   0.  -4.]
 [  0.  -2.  -2.   1.  -1.   6.]
 [  1.  -1.   6.   1.   1. -12.]]
[ 1.  1.  1. -1.  1. -0.]
[[ 1.          0.          0.          0.5        -0.83333333  1.33333333]
 [-0.          1.          0.         -0.5         0.16666667 -0.66666667]
 [ 0.          0.          1.          0.          0.33333333 -2.33333333]]
[ 0.          0.          0.         -1.          1.33333333  1.66666667]
0
4 0
[[-1.2 -0.  -0.  -0.6  1.  -1.6]
 [ 0.2  1.   0.  -0.4  0.  -0.4]
 [ 0.4  0.   1.   0.2  0.  -1.8]]
[ 1.6  0.   0.  -0.2  0.   3.8]
1
0 1
[[ 0.  6.  0. -3.  1. -4.]
 [ 1.  5.  0. -2.  0. -2.]
 [ 0. -2.  1.  1.  0. -1.]]
[ 0. -8.  0.  3.  0.  7.]
2
3 2
[[ 0.00000000e+00 -1.77635684e-15  3.00000000e+00  0.00000000e+00
   1.00000000e+00 -7.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  2.00000000e+00  0.00000000e+00
   0.00000000e+00 -4.00000000e+00]
 [ 0.00000000e+00 -2.00000000e+00  1.00000000e+00  1.00000000e+00
   0.00000000e+00 -1.00000000e+00]]
[ 0

In [180]:
#TODO

task5 = Task(
    [-1, 4, -3, 10], TaskType.MIN,
    [
        Inequality([1, 1, -1, -10], Signs.EQUAL, 0),
        Inequality([1, 14, 10, -10], Signs.EQUAL, 11),
    ]
)

result_5 = solve(
    task5,
    [0, 0, 1, 1],
    [0, 0, 1, 1]
)

check(result_5, -4.0)


2
(2, 5)
[[  1.   1.  -1. -10.   0.]
 [  1.  14.  10. -10. -11.]]
[  1.  -4.   3. -10.  -0.]
[[ 0.          1.18181818  1.          0.         -1.        ]
 [-0.1        -0.21818182 -0.          1.          0.1       ]]
[ 0.         -9.72727273  0.          0.          4.        ]
complete
Result:  -4.0


In [181]:
task_6 = Task(
    [-1, 5, 1, -1], TaskType.MIN,
    [
        Inequality([1, 3, 3, 1], Signs.LESS, 3),
        Inequality([0, 2, 3, -1], Signs.LESS, 4),
    ]
)

result_6 = solve(
    task_6,
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1]
)

check(result_6, 3)


2
(2, 7)
[[ 1.  3.  3.  1.  1.  0. -3.]
 [ 0.  2.  3. -1.  0.  1. -4.]]
[ 1. -5. -1.  1. -0. -0. -0.]
[[ 1.  3.  3.  1.  1.  0. -3.]
 [ 0.  2.  3. -1.  0.  1. -4.]]
[ 1. -5. -1.  1.  0.  0.  0.]
0
0 0
[[ 1.  3.  3.  1.  1.  0. -3.]
 [ 0.  2.  3. -1.  0.  1. -4.]]
[ 0. -8. -4.  0. -1.  0.  3.]
complete
Result:  -3.0
not equal. expected: 3, got: -3.0


In [182]:
task_7 = Task(
    [-1, -1, 1, -1, 2], TaskType.MIN,
    [
        Inequality([3, 1, 1, 1, -2], Signs.EQUAL, 10),
        Inequality([6, 1, 2, 3, -4], Signs.EQUAL, 20),
        Inequality([10, 1, 3, 6, -7], Signs.EQUAL, 30)
    ]
)

result_7 = solve(
    task_7,
    [0, 0, 1, 2, 1],
    [0, 0, 1, 1, 1]
)

check(result_7, -10)


3
(3, 6)
[[  3.   1.   1.   1.  -2. -10.]
 [  6.   1.   2.   3.  -4. -20.]
 [ 10.   1.   3.   6.  -7. -30.]]
[ 1.  1. -1.  1. -2. -0.]
[[  1.   0.   1.   0.   0. -10.]
 [  0.  -1.   0.   1.   0.   0.]
 [ -1.  -1.  -0.  -0.   1.  -0.]]
[  0.   0.   0.   0.   0. -10.]
complete
Result:  10.0
not equal. expected: -10, got: 10.0


In [183]:
# import sympy as sy
# a = np.array([sy.Rational(1), sy.Rational(2), sy.Rational(3)], dtype=sy.Rational)
# b = a / 17 / 19 * 17
# print(b)
# print(sy.Rational('6/5') + 1)